#**`NB2_Transformer_Model : Fine-Tuning a Transformer Model`**

In [1]:
#mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Reading the data from google drive
data=pd.read_csv('gdrive/MyDrive/SEA820_DataSets/AI_Human.csv')

In [4]:
#use a copy so that the changes made to the copy does not affect the original data set
#this is a good coding practice
df = data.copy()

In [5]:
#print head
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [6]:
df = df.rename(columns={'generated': 'label'})
df['label'] = df['label'].astype('int64')

In [7]:
df.head()

,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [29]:
df_sample = (
    df.groupby('label', group_keys=False)
      .apply(lambda g: g.sample(frac=5000/len(df), random_state=42))
      .reset_index(drop=True)
)

/tmp/ipython-input-1268345527.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=5000/len(df), random_state=42))


In [30]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.3+ KB


In [31]:
#using a smaller sample for faster model training maintaining the ratio of ai vvd human entries
df_sample['label'].value_counts()

,count
label,
0,3138
1,1862


In [32]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
tokenizer_distilbert = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [33]:
#Load your sentiment dataset into a Hugging Face Dataset object
from datasets import Dataset
#(e.g., from a Pandas DataFrame or CSV file using datasets.Dataset.from_pandas() or load_dataset('csv', ...)).
dataset = Dataset.from_pandas(df_sample)

In [34]:
# Perform a train-test split using the dataset's train_test_split method. Load the tokenizer for a suitable pre-trained model intended for sequence classification (e.g., distilbert-base-uncased).
train_test_split = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [35]:
#Create a function that takes a batch of examples from the dataset and tokenizes the text using the loaded tokenizer.
def tokenize(batch):
    return tokenizer_distilbert(batch['text'], padding='max_length', truncation=True, max_length=128)

In [36]:
#Ensure padding and truncation are handled (padding='max_length', truncation=True, max_length=... e.g., 128 or 256).
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [37]:
#Load the pre-trained model architecture suitable for sequence classification using AutoModelForSequenceClassification.from_pretrained().
from transformers import AutoModelForSequenceClassification

#Pass the model name (e.g., distilbert-base-uncased) and specify the number of labels in your dataset (num_labels=2 for binary sentiment).
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
#Instantiate TrainingArguments from the transformers library.
from transformers import TrainingArguments

In [39]:
#Specify essential arguments: output_dir: A directory to save the model checkpoints and results.
training_args = TrainingArguments(
    output_dir="./finetune-checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate = 2e-5,
    weight_decay=0.01,
    #evaluation_strategy="epoch", #this was causing error
    logging_dir="./logs",
    logging_steps=10,
)

In [40]:
#Define a function compute_metrics(eval_pred) that takes evaluation predictions, calculates desired metrics (e.g., accuracy, F1 using sklearn.metrics), and returns them as a dictionary.
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

#The predictions will be logits, so you'll need to apply argmax to get the predicted labels.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions)
    }

In [41]:
#Instantiate the Trainer class, passing the model, training arguments, training dataset, evaluation dataset, tokenizer, and the compute_metrics function.

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer_distilbert,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2131909470.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
#Start the fine-tuning process by calling trainer.train().
trainer.train()

Step,Training Loss
10,0.650600
20,0.585300
30,0.469100
40,0.390300
50,0.326300
60,0.253400
70,0.268700
80,0.201400
90,0.252500
100,0.145900


TrainOutput(global_step=250, training_loss=0.22927211332321168, metrics={'train_runtime': 2999.8265, 'train_samples_per_second': 1.333, 'train_steps_per_second': 0.083, 'total_flos': 132467398656000.0, 'train_loss': 0.22927211332321168, 'epoch': 1.0})

In [43]:
#Start the fine-tuning process by calling trainer.train().
metrics = trainer.evaluate()

#Report the final evaluation metrics obtained.
print(metrics)

{'eval_loss': 0.08955828845500946, 'eval_accuracy': 0.97, 'eval_f1': 0.9605263157894737, 'eval_runtime': 220.1059, 'eval_samples_per_second': 4.543, 'eval_steps_per_second': 0.286, 'epoch': 1.0}
